# 🏥 Healthcare Insurance Cost Analysis  
## 📊 Notebook 09 – Individuals with more children (dependents) may experience slightly higher insurance costs..

| Feild | Description |
|-------|-------------|
|**Author:** | Robert Steven Elliott  |
|**Course:** | Code Institute – Data Analytics with AI Bootcamp |  
|**Project Type:** | Individual Formative Project  | 
|**Date:** | December 2025  |


$$
    \begin{align}
        H_{0}&: \text{Individuals with more children will not experience slightly higher insurance costs.} \\
        H_{1}&: \text{Individuals with more children may experience slightly higher insurance costs.}
    \end{align}
$$

---

## Change Working Directory

In [1]:
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT))
print("✅ Working directory set to project root:", PROJECT_ROOT)

✅ Working directory set to project root: /home/robert/Projects/health-insurance-cost-analysis


---

## Import Libraries and Dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from utils.data_handler import load_data, data_overview, clean_data

pd.set_option('display.max_columns', None)
sns.set_theme(style="whitegrid")

input_path = PROJECT_ROOT / "data" / "final" / "insurance_final.csv"
figure_path = PROJECT_ROOT / "figures"

if not figure_path.exists():
    figure_path.mkdir(parents=True, exist_ok=True)
    print(f"✅ Created figure directory at: {figure_path}")

df = load_data(input_path)
df = clean_data(df, categorical_cols=['sex', 'smoker', 'region', 'bmi_category', 'age_group', 'family_size_category'])
data_overview(df)
print("✅ Data loaded successfully.")
df.head()

DataFrame Shape: (1337, 10)

Data Types:
 age                        int64
sex                     category
bmi                      float64
children                   int64
smoker                  category
region                  category
charges                  float64
bmi_category            category
age_group               category
family_size_category    category
dtype: object

Missing Values:
 age                     0
sex                     0
bmi                     0
children                0
smoker                  0
region                  0
charges                 0
bmi_category            0
age_group               0
family_size_category    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   age                   1337 non-null   int64   
 1   sex                   1337 non-null   category
 2   bmi

,age,sex,bmi,children,smoker,region,charges,bmi_category,age_group,family_size_category
0,19,female,27.900,0,yes,southwest,16884.92400,Overweight,18-25,No Children
1,18,male,33.770,1,no,southeast,1725.55230,Obese,18-25,Small Family
2,28,male,33.000,3,no,southeast,4449.46200,Obese,26-35,Medium Family
3,33,male,22.705,0,no,northwest,21984.47061,Normal,26-35,No Children
4,32,male,28.880,0,no,northwest,3866.85520,Overweight,26-35,No Children


## Family Size vs Charges

In [3]:
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=[
        "Number of Children vs Insurance Charges",
        "Average Insurance Charges by Family Size Category"
    ]
)

scatter_fig = px.box(
    df,
    x='children',
    y='charges',
    color='children',
    points='all'
)

# Add scatter traces, only show legend for the first trace of each color
added_legends = set()
for trace in scatter_fig['data']:
    # Only show legend if we haven't added this name yet
    if trace.name in added_legends:
        trace.showlegend = False
    else:
        trace.showlegend = True
        added_legends.add(trace.name)
    fig.add_trace(trace, row=1, col=1)

# -----------------------------
# Grouped bar chart
# -----------------------------
charges_avg = df.groupby('family_size_category', observed=False)['charges'].mean().reset_index()
charges_avg.rename(columns={'charges': 'charges'}, inplace=True)
barchart_fig = px.bar(
    charges_avg,
    x='family_size_category',
    y='charges',
    labels={'family_size_category': 'Family Size Category', 'charges': 'Average Charges'},
    category_orders={'family_size_category': ['No Children', 'Small Family', 'Medium Family', 'Large Family']}
)

for trace in barchart_fig['data']:
    trace.showlegend = False # No legend needed for single variable
    fig.add_trace(trace, row=1, col=2)

# -----------------------------
# Layout adjustments
# -----------------------------
fig.update_layout(
    showlegend=True,
    legend=dict(
        title=dict(text="Family Size"), 
        x=1.05, 
        y=1
    ),
    height=500,
    width=1200,
    title_text=" H6: Number of Children vs Charges",
)
fig.write_html(figure_path / "children_charges_family_size_analysis.html")
fig.write_image(figure_path / "children_charges_family_size_analysis.png")
fig.show()


### Observation:
- Average charges increase slightly with more children but the relationship is weak.
- H6 is only partly supported — dependents may have a small additive cost.